# Re_entreno

In [30]:
import torch
torch.cuda.is_available()

False

In [31]:
# Para windows con la CPU:
!pip3 install torch torchvision torchaudio 

# Para Windows con CUDA:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

# Para windows con la CPU:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Para Windows con CUDA:
# !pip3 install torch torchvision torchaudio

# Es posible que la mayoría de dependencias ya esten instaladas ^^

In [32]:
!git clone https://github.com/ultralytics/yolov5
# Se deberia haber creado una carpeta en nuestro area de trabajo
# Si ya lo teniamos saldra el siguiente error =>fatal: destination path 'yolov5' already exists and is not an empty directory.

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [33]:
!cd yolov5 & pip install -r requirements.txt

In [34]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2 #Para abrir webcam

# Esta línea de código establece una variable de entorno llamada KMP_DUPLICATE_LIB_OK con el valor "True", lo que significa que permite la carga de bibliotecas duplicadas en el entorno de Python. Esto es útil para algunas bibliotecas de Machine Learning que usan OpenMP para acelerar el cómputo.
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

***
En la siguiente celda se escogen 
***

In [35]:
import os
import random
import shutil

random.seed(15)

src = "C:\\Users\\shirk\\OneDrive\\Documentos\\Master\\Proyecto_II\\IA\\Re_entrenamiento_100_Imagenes\\data\\YoloV5\\"
dest_Image = "C:\\Users\\shirk\\OneDrive\\Documentos\\Master\\Proyecto_II\\IA\\Re_entrenamiento_100_Imagenes\\data\\images\\"
dest_Label = "C:\\Users\\shirk\\OneDrive\\Documentos\\Master\\Proyecto_II\\IA\\Re_entrenamiento_100_Imagenes\\data\\labels\\"

lista_aleatorios_rellenada = []

for n in range(100):
  aleatorio = random.randint(0,900)
  aleatorio_rellenado = str(aleatorio).zfill(5)
  src_img = src + str(aleatorio_rellenado) + ".jpg"
  if os.path.exists(src_img):
    if aleatorio_rellenado not in lista_aleatorios_rellenada:
      lista_aleatorios_rellenada.append(aleatorio_rellenado)

print(lista_aleatorios_rellenada)



for num in lista_aleatorios_rellenada:
  src_img = src + str(num) + ".jpg"
  dest_img = dest_Image + str(num) + ".jpg"
  shutil.copy2(src_img, dest_img)
  print(dest_img)

for num in lista_aleatorios_rellenada:
  src_lab = src + str(num) + ".txt"
  dest_lab = dest_Label + str(num) + ".txt"
  shutil.copy2(src_lab, dest_lab)
  print(dest_lab)

['00011', '00037', '00161', '00244', '00017', '00056', '00824', '00697', '00150', '00376', '00245', '00119', '00346', '00477', '00725', '00287', '00401', '00269', '00352', '00234', '00211', '00895', '00363', '00816', '00320', '00228', '00312', '00744', '00523', '00430', '00236', '00466', '00860', '00501', '00080', '00472', '00818', '00368', '00449', '00322', '00710', '00661', '00461', '00414', '00795', '00884', '00829', '00067', '00502', '00858', '00019', '00200', '00143', '00176', '00628', '00803', '00707', '00864', '00736', '00293', '00542', '00155', '00876', '00102', '00404', '00048', '00075', '00479', '00742', '00302', '00411', '00514', '00237', '00183', '00556', '00512', '00362', '00240', '00317', '00318', '00168']
C:\Users\shirk\OneDrive\Documentos\Master\Proyecto_II\IA\Re_entrenamiento_100_Imagenes\data\images\00011.jpg
C:\Users\shirk\OneDrive\Documentos\Master\Proyecto_II\IA\Re_entrenamiento_100_Imagenes\data\images\00037.jpg
C:\Users\shirk\OneDrive\Documentos\Master\Proyecto_I

***
**`Entrenar un modelo desde cero`**
***

In [36]:
import uuid   # Unique identifier
import time

In [37]:
!git clone https://github.com/tzutalin/labelImg

fatal: destination path 'labelImg' already exists and is not an empty directory.


In [38]:
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

In [39]:
%run -i "labelImg\labelImg.py"

Ahora tenemos que crear el archivo dataset.yaml, que contiene las categorias de nuestros pesos y las rutas de nuestras imagenes de entrenamiento, test y validación.

In [40]:
f= open("yolov5/data/cien_images_señales.yaml","w+")

f.write("path: ../data \ntrain: images \nval: images \n\nnc: 4 \n\nnames: ['prohibitory', 'danger', 'mandatory', 'other']")

f.close()

In [41]:
!cd yolov5 && python train.py --img 640 --batch 16 --epochs 500 --data cien_images_señales.yaml --weights yolov5s.pt  --workers 2

In [ ]:
# LOGGER.info(( '%10s ' * 7) % ('Epoch', 'gpu_mem', 'box', 'obJ', 'cls', 'labels', 'img_size'))
# if RANK in [-1, 0]:
#     pbar = tqdm(pbar, total=nb, bar_format='{1_bar}{bar:10}{r_bar}{bar:-10b}') #progress_bar

***
**`Cargamos nuestro modelo`**
***

In [ ]:
modelo_100_señales = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp10/weights/best.pt', force_reload=True)